In [2]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import re
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [48]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36


<ipython-input-48-049692b84c7f>:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [8]:
driver.get('https://www.noon.com/saudi-ar/p-11433/?limit=150')
list_products = []

In [12]:

r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")
time.sleep(1)
products = soup.find_all('div', {'class': 'productContainer'})

len(products)
for produc in products:
    list_products.append(produc.find('a')['href'])
len(list_products)

582

In [61]:

r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")
time.sleep(1)
products = soup.find_all('div', {'class': 'productContainer'})
cats = soup.find('div', {'class': 'sc-loqmag-1 cLHial'}).find_all('a')
len(cats)
cat1 = cats[0].text
cat1

'المنزل والمطبخ'

In [38]:
def extract_ele(name, soup):
    elements = soup.find('div', {'class': 'sc-1oxmzfx-1 jpyJdX'}).find_all('td')
    for ele in elements:
        if name in ele.text:
            return ele.next_element.next_element.text.strip()
        

In [50]:
def extract_ele(name, soup):
    elements = soup.find('div', {'class': 'sc-1oxmzfx-1 jpyJdX'}).find_all('td')
    for ele in elements:
        if name in ele.text:
            return ele.next_element.next_element.text.strip()
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Noon/Noon_model_product.xlsx')
for i, url in enumerate(list_products):
    print('Count: ', i)
    url = 'https://www.noon.com' + url
    print('URL: ', url)
    driver.get( url)
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, "html.parser")
    time.sleep(0.5)
    try:
        sku = soup.find('div', {'class': 'modelNumber'}).text.strip()
    except:
        continue
    name = soup.find('h1', {'class': 'sc-1vbk2g0-12 xaGRV'}).text.strip()
    try:
        price = soup.find('div', {'class': 'priceWas'}).text.replace('ر.س', '').strip()
        special_price = soup.find('div', {'class': 'priceNow'}).text.replace('شاملاً ضريبة القيمة المضافة', '').replace('ر.س', '').replace('', '').strip()
    except:
        price = soup.find('div', {'class': 'priceNow'}).text.replace('شاملاً ضريبة القيمة المضافة', '').replace('ر.س', '').strip()
        special_price = ''
    description = soup.find('div', {'class': 'sc-xf6b4m-0 iINLuG'}).text
        
    try:
        click_btn = driver.find_element_by_xpath('//div[contains(text(),"عرض الموصفات بالكامل")]')
        click_btn.click()
        r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
        soup = BeautifulSoup(r, "html.parser")
    except:
        pass
    raw_materials = extract_ele('المادة', soup)
    free_colors = extract_ele('اسم اللون', soup)
    products_size = extract_ele('الحجم', soup)
    sku = extract_ele('رقم الموديل', soup)
    manufacturer = extract_ele('بلد المنشأ', soup)
    number_pieces = extract_ele('عدد القطع', soup)
    ts_dimensions_width = extract_ele('عرض/عمق المنتج', soup)
    ts_dimensions_length = extract_ele('طول المنتج', soup)
    ts_dimensions_height = extract_ele('ارتفاع المنتج', soup)
    set_include = extract_ele('الطقم يتضمن', soup)
    weight = extract_ele('وزن المنتج', soup)
    images = soup.find('div', {'class': 'sc-1wxm9lw-4 iFrHQU'}).find_all('img')
    len(images)
    cats = soup.find('div', {'class': 'sc-loqmag-1 cLHial'}).find_all('a')
    len(cats)
    cat1 = cats[0].text
    cat2 = cats[1].text
    try:
        cat3 = cats[-1].text
    except:
        cat3 = ''
    
    list_images = [img['src'] for img in images]
    base_images = list_images[0]
    additionnel_images = ','.join(list_images[1:])

    data = {
        
        'sku': sku,
        'name': name,
        'link_url': url,
        'price': price,
        'special_price': special_price,
        'description': description,
        'free_colors': free_colors,
        'ts_dimensions_width':ts_dimensions_width,
        'ts_dimensions_length':ts_dimensions_length,
        'ts_dimensions_height':ts_dimensions_height,
        'weight': weight,
        'products_size': products_size,
        'set_include': set_include,
        'number_pieces': number_pieces, 
        'categories1': cat1,
        'categories2': cat2,
        'categories3': cat3,
        'raw_materials': raw_materials,
        'manufacturer': manufacturer,
        'base_images': base_images,
        'additionnel_images': additionnel_images,
    }
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('Noon-update1.xlsx')



Count:  0
URL:  https://www.noon.com/saudi-ar/12-piece-multi-purpose-drinking-glass-set-clear/N46559239A/p/?o=d50bd603c39e6e7d
Count:  1
URL:  https://www.noon.com/saudi-ar/12-piece-coffee-cup-and-saucer-set-white-6x200ml/N14022922A/p/?o=bf6c68dcf1b570aa
Count:  2
URL:  https://www.noon.com/saudi-ar/3-piece-bakeware-pan-set-clear-small-1x1-6-medium-1x2-2-large-1x2-9l/N21584994A/p/?o=f2072bbc45f62f4f
Count:  3
URL:  https://www.noon.com/saudi-ar/non-stick-round-fry-pan-beige-black-18cm/N40132979A/p/?o=e2b60335b3e7d8ea
Count:  4
URL:  https://www.noon.com/saudi-ar/4-piece-serving-plates-white-4x10inch/N45029927A/p/?o=f4c236c2a14f0ace
Count:  5
URL:  https://www.noon.com/saudi-ar/6-piece-serving-bowl-set-clear-345ml/N38470148A/p/?o=b89a0f832875035d
Count:  6
URL:  https://www.noon.com/saudi-ar/8-piece-mini-side-dish-white-4inch/N45029931A/p/?o=e051f7ec87cf331d
Count:  7
URL:  https://www.noon.com/saudi-ar/4-piece-soup-bowls-white-6inch/N45029939A/p/?o=c8a5d1975aa1c01f
Count:  8
URL:  http

In [22]:
df.columns

Index(['sku', 'name', 'link_url', 'price', 'special_price', 'description',
       'free_colors', 'ts_dimensions_width', 'ts_dimensions_length',
       'ts_dimensions_height ', 'weight', 'products_size', 'set_include',
       'number_pieces', 'categories1', 'categories2', 'categories3',
       'raw_meterials', 'manufacturer', 'base_images', 'additionnel_images',
       'ts_dimensions_height', 'raw_materials'],
      dtype='object')

In [23]:
categories1 = df['categories1'].unique().tolist()
categories2 = df['categories2'].unique().tolist()
categories3 = df['categories3'].unique().tolist()


In [29]:
for i in range(2, 7):
    categories2.append('')

In [34]:
categories2

['المطبخ وأدوات الطعام', 'أنشطة الأماكن الخارجية', '', '', '', '', '']

In [46]:
list_cats = []

list_cats = ({'categories1': categories1, 'categories2': categories2, 'categories3': categories3})

In [47]:
list_toto = pd.DataFrame(list_cats)
list_toto.to_excel('Noon-categories.xlsx')

In [18]:
click_btn = driver.find_element_by_xpath('//div[contains(text(),"عرض الموصفات بالكامل")]')
click_btn.click()

In [2]:
cats = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Noon/Noon-update1.xlsx')


In [4]:
categories1 = cats['categories1'].unique().tolist()
len(categories1)

2

In [5]:
categories2 = cats['categories2'].unique().tolist()
len(categories2)

2

In [9]:
for i in range(2, 28):
    categories2.append('')


In [10]:
len(categories2)

28

In [6]:
categories3 = cats['categories3'].unique().tolist()
len(categories3)

28

In [11]:
data = {
    'categories1': categories1,
    'categories2': categories2,
    'categories3': categories3,
}

In [14]:
df = pd.DataFrame(data)
df.to_excel('Noon-categories.xlsx')

In [43]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Noon/Noon-update1.xlsx')


In [25]:
def clean_string(toto):
    return toto[:-1]

In [44]:
df['price'] = df['price'].str.rstrip(' .').str.strip()
df['special_price'] = df['special_price'].str.rstrip(' .').str.strip()

In [45]:
df['price'] = df['price'].str.rstrip(' .').str.strip()
df['special_price'] = df['special_price'].str.rstrip(' .').str.strip()
df['price'] = df['price'].str.rstrip('.‏()').str.strip()
df['special_price'] = df['special_price'].str.rstrip('.‏()').str.strip()

df['price'] = pd.to_numeric(df['price'])
df['special_price'] = pd.to_numeric(df['special_price'])


In [48]:
df['special_price'] 

0       39.0
1       34.0
2       49.0
3       23.0
4       19.0
       ...  
568    289.0
569     34.0
570     69.0
571     89.0
572     89.0
Name: special_price, Length: 573, dtype: float64

In [24]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Noon/Noon-update2.xlsx')
cat1 = df['categories1'].unique().tolist()
cat2 = df['categories2'].unique().tolist()
cat3 = df['categories3'].unique().tolist()

In [28]:
len(cat1), len(cat2), len(cat3)

(234, 234, 234)

In [27]:
for i in range(24, 234):
    cat2.append('')

In [29]:
data = {
    'categories1': cat1,
    'categories2': cat2,
    'categories3': cat3,
}

df1 = pd.DataFrame(data)
df1.to_excel('Noon-categories-.xlsx')

In [22]:
df.loc[df['categories3'].str.contains('https:'), 'categories1']

4443     600ml: 13X9.5X12cmPackage weight: 350ml: 220g
Name: categories1, dtype: object

In [36]:
df1.drop(df1.loc[df1['categories3'].str.contains('https:')].index, inplace=True)

In [1]:
def extract_ele(name, soup):
    elements = soup.find('div', {'class': 'sc-215afa5f-2 qQwBS'}).find_all('td')
    for ele in elements:
        if name in ele.text:
            return ele.next_element.next_element.text.strip()

In [2]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

NameError: name 'driver' is not defined

In [ ]:
df1.to_excel('Noon-categories-1.xlsx')